In [1]:
import os, collections
import numpy as np
import pandas as pd

from mpire import WorkerPool

RAW_PATH = "/data2/zhoujb/project/cowpea_project/rawData/"
SNP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/snpDir/"

In [2]:
def run_getrawSNP(chr_name, from_bp, end_bp):
    
    chr_snp_file = pd.read_table(os.path.join(RAW_PATH, "{}.snplist".format(chr_name)))
    chr_snp_file = chr_snp_file.sort_values(by=["Pos"])

    keep_index = []
    for index in chr_snp_file.index:
        tmp_pos = int(chr_snp_file.loc[index, "Pos"])
        if tmp_pos > end_bp:
            break
        elif tmp_pos < from_bp:
            continue
        else:
            # Remove snp threshold 20%
            snp_list = chr_snp_file.loc[index].values[2:]
            count_d = collections.Counter(snp_list)
            if (count_d["A"] + count_d["T"] + count_d["C"] + count_d["G"]) / len(snp_list) >= 0.8:
                keep_index.append(index)

    if len(keep_index) > 10:
        # Remove sample
        tmp_data = chr_snp_file.loc[keep_index].copy()
        rm_sample = []
        for col in tmp_data.columns[2:]:
            tmp_d = collections.Counter(tmp_data[col].values)
            if (tmp_d["A"] + tmp_d["T"] + tmp_d["C"] + tmp_d["G"]) / sum(tmp_d.values()) < 0.8:
                rm_sample.append(col)

        tmp_data = tmp_data.drop(columns=rm_sample)
        tmp_data = tmp_data.rename(columns={"#Chr":"Chr"})
        tmp_data.to_csv(os.path.join(SNP_PATH, "{}_{}_{}.txt".format(chr_name, from_bp, end_bp)), sep="\t", index=0)
    return

if __name__ == "__main__":
    input_list = []
    region_info = pd.read_table(os.path.join(RAW_PATH, "XP-CLR_region.txt"))
    for index in region_info.index:
        Chr = region_info.loc[index, "Chromosome"]
        Start = region_info.loc[index, "Region start (bp)"]
        End = region_info.loc[index, "Region end (bp)"]
        input_list.append((Chr, Start, End))

    with WorkerPool(n_jobs=48) as pool:
        pool.map(run_getrawSNP, input_list, progress_bar=True)

    print("ALL DONE!")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [02:15<00:00,  3.86it/s]

ALL DONE!


In [3]:
len(input_list)

948